<a href="https://colab.research.google.com/github/xpdlaldam/nlp/blob/master/HF_audio_course.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
1. Background
By nature, a sound wave is a continuous signal, meaning it contains an infinite number of signal values in a given time.
This poses problems for digital devices which expect finite arrays.
To be processed, stored, and transmitted by digital devices, the continuous sound wave needs to be converted into a series of discrete values,
known as a digital representation.

2. Sampling and sampling rate
  2-1. Sampling:
    - the process of measuring the value of a continuous signal at fixed time steps. The sampled waveform is discrete, since it contains a finite number of signal values at uniform intervals.
  2-2. Sampling rate (also called sampling frequency)
    - the number of samples taken in one second and is measured in hertz (Hz)
    - how often the samples are taken

3. Resampling
Since sequences are different for audio examples at different sampling rates, it will be challenging for models to generalize between sampling rates. Resampling is the process of making the sampling rates match, and is part of preprocessing the audio data.

4.
"""

In [7]:
import librosa
array, sampling_rate = librosa.load(librosa.ex("trumpet"))

In [8]:
array

array([-1.4068222e-03, -4.4607258e-04, -4.1098078e-04, ...,
        7.9623051e-06, -3.0417003e-05,  1.2765067e-05], dtype=float32)

In [9]:
sampling_rate

22050